### Prepare data for cell phase classification model training

### 一、准备和组织数据

用于识别细胞周期的数据为单个细胞图像文件，其大小取决于细胞毒大小，但是您只需要提供标注好的json文件，以及相应的原始tif图像文件或者tif图像序列，就能够自动生成用于训练的数据集。您的原始tif文件请存放于raw/目录下，tif序列文件请存放于tif/目录下。您应该将标注文件和原始图像按照以下方式组织：


    dataset1/：
    |    |
    |    |__raw/
    |    |
    |    |__tif/
    |    |   |__dic/
    |    |       ...
    |    |   |__mcy/
    |    |       ...
    |    |__dataset1.json
    |
    dataset2/：
    |    |__raw/
    |
    |    |__tif/
    |       |__dic/
    |           ...
    |       |__mcy/
    |           ...
    |    |__dataset2.json
     ...
 
 其中每一个dataset对应着一个json标注文件。
 
 
 在准备好以上数据后，就可以调用`CCDeep/tools/generate_dataset.py`中的`generate()`函数生成训练数据。该函数默认只需要提供上述`dataset`的路径，例如：`generate('~/data/dataset1')`。
 
 
 
 
 此函数会将生成的数据保存在每个`dataset`的同级目录下的`train_dataset`文件夹中。您可以通过给`generate()`函数传递`save_train_dataset`参数来改变这个文件夹名，但不建议这么做。
 
 执行完之后，您的`dataset`目录应该是如下结构：
 
    dataset1/：

        |__tif/
        |   |__dic/
        |       ...
        |   |__mcy/
        |       ...
        |__dataset1.json
        |
        |__train_dataset/
                   |__dic/
                   |    |__G/
                   |    |__M/
                   |    |__S/
                   |
                   |__mcy/
                   |    |__G/
                   |    |__M/
                   |    |__S/
        
您可以将每个`dataset`中的`train_dataset`合并在一个文件夹中，只需要保证结构不乱即可。`train_dataset`中的数据即是用于最后训练和验证的数据。


---------
                   

### 二、数据增强

您可以选择在训练之前进行数据增强，这样能够节省训练时的内存需求，同时加快训练速度，但是会占用很大的存储空间，如果您的硬盘足够大，请选用此方式；否则，请在`config.py`中设置`AUGMENTATION_IN_TRAINING=True`。

当您选择在训练前进行数据增强，您需要调用`CCDeep/tools/augmentation.py`中的`augment()`函数。该函数有两个参数，`dic`即上述`train_dataset`中的`dic`路径名，`mcy`即相应的`mcy`路径名。例如`augment(dic='~/data/dataset1/train_dataset/dic', mcy='~/data/dataset1/train_dataset/mcy')`。

该函数会在`train_datasset`目录下生成相应的`augment_dic`和`augment_mcy`文件夹，其中存放着增强后的所有数据。

此时您的文件结构应该是这样：

    dataset1/：

        |__tif/
        |   |__dic/
        |       ...
        |   |__mcy/
        |       ...
        |__dataset1.json
        |
        |__train_dataset/
                   |__dic/
                   |    |__G/
                   |    |__M/
                   |    |__S/
                   |
                   |__mcy/
                   |    |__G/
                   |    |__M/
                   |    |__S/
                   |
                   |__augment_dic/
                   |        |__G/
                   |        |__M/
                   |        |__S/
                   |
                   |__augment_mcy/
                   |        |__G/
                   |        |__M/
                   |        |__S/

当您选择在训练开始之前进行数据增强后，所有的训练和验证数据集均来自于`augment_dic`和`augment_mcy`。

----------

### 三、划分训练集，验证集和测试集

训练模型需要提供训练集，验证集和测试集，由于数据量大，无法一次性读入，我们选择了先划分再时输入，因此您需要提前将数据集进行划分。您需要调用`CCDeep/tools/split_dataset.py`中的`split(dataset_mcy_path, save_mcy_path)`方法，其中`dataset_mcy_path`是上述`augment_mcy`的路径，`save_mcy_path`是分割后`mcy`训练集，验证集和测试集保存的目录。相应的`dic`数据会同步分割，不用重复执行。`save_mcy_path`可以是任何有效路径，但我们仍建议您将其设置为与`dataset_mcy_path`同级目录，如果您这么做了，假设您的`dataset_mcy_path`为`~/data/dataset1/train_dataset/augment_mcy`，`save_mcy_path`为`~/data/dataset1/train_dataset/train_mcy`,那么分割后您的文件结构应该如下：



    dataset1/：

        |__tif/
        |   |__dic/
        |       ...
        |   |__mcy/
        |       ...
        |__dataset1.json
        |
        |__train_dataset/
                   |__dic/
                   |    |__G/
                   |    |__M/
                   |    |__S/
                   |
                   |__mcy/
                   |    |__G/
                   |    |__M/
                   |    |__S/
                   |
                   |__augment_dic/
                   |        |__G/
                   |        |__M/
                   |        |__S/
                   |
                   |__augment_mcy/
                   |        |__G/
                   |        |__M/
                   |        |__S/
                   |
                   |__train_dic/
                   |        |__test/
                   |        |__train/
                   |        |__valid
                   |
                   |__train_mcy/
                   |        |__test/
                   |        |__train/
                   |        |__valid/
 
 在`config.py`中，`dataset_dir_mcy_20x`即上述`~/data/dataset1/train_dataset/train_mcy`， `dataset_dir_dic_20x`即`~/data/dataset1/train_dataset/train_dic`。